In [1]:
from sleepmate.executor import *
from sleepmate.act import *
from sleepmate.helpful_scripts import *
from sleepmate.diary import *
from sleepmate.goal import *
from sleepmate.seeds import *
from sleepmate.structured import *
from sleepmate.history import *
from sleepmate.config import *
from sleepmate.history import *
from sleepmate.knowledge import *
from sleepmate.user import *
from sleepmate.meet import *
from sleepmate.db import *

In [2]:
username="ivy0484"
username="cck197"

In [3]:
x = X(username=username, hello=None)

In [7]:
x("hi");

2023-10-29 10:10:37 INFO     sleepmate.executor 653857366f561d27ec50ea2a None -> all_for_now


```
That's all I have for now. Let me know if you have any questions, I'm happy to help. Otherwise, I'll reach out to check in with you later.
```

In [8]:
x.get_next_goal()

all_for_now

In [11]:
x.goals["GOAL_OPTIONS"]#["foo"]#["bar"]; d

{'all_for_now': {'rigid': True}}

In [10]:
d.get("all_for_now", {}).get("rigid", False)

True

In [6]:
x.goals["GOAL_OPTIONS"][str(x.get_next_goal())]["rigid"]

True

In [5]:
x.clear_chat_history(N=9)

2023-10-29 10:48:06 INFO     sleepmate.executor clear_chat_history: deleting ids_to_delete=[ObjectId('653e93f2ad693924ec8e2c0f'), ObjectId('653e93f2ad693924ec8e2c0e'), ObjectId('653e93d5ad693924ec8e2c0c'), ObjectId('653e93d5ad693924ec8e2c0b'), ObjectId('653e93b5ad693924ec8e2c09'), ObjectId('653e93b5ad693924ec8e2c08'), ObjectId('653e6eb7f459e342109cfb6d'), ObjectId('653e6eb7f459e342109cfb6c'), ObjectId('653e64379e8cfa8e1dbd4c53')]


In [6]:
list(reversed(list(enumerate(x.memory.chat_memory.messages))))

[(332, HumanMessage(content='yep')),
 (331,
  AIMessage(content="Great job, Chris! Let's summarise your SEEDS tasks for the 28th of October, 2023:\n\n- Got outside and walked by 9am\n- Got in the sauna at 8pm\n- Avoided chocolate after 7pm\n- Did your PT exercises whilst your coffee was brewing\n- Drank three glasses of water\n- Did the open focus exercise\n- Wrote down three things you're grateful for\n- Sent one message to a friend\n\nYou didn't manage to:\n\n- Walk around the block after lunch\n- Do 5 deadlifts after cycling, surfing or jujitsu\n- Limit yourself to no more than 3/4 bar of chocolate\n\nDoes everything look correct?")),
 (330, HumanMessage(content='did 1-3, 4, 8, 9-11')),
 (329,
  AIMessage(content="Excellent, Chris. Let's go through each of the SEEDS pillars and see what you managed to do yesterday, the 28th of October, 2023.\n\nStarting with Sleep:\n1. Did you manage to get outside and walk by 9am?\n2. Did you get in the sauna at 8pm?\n3. Did you avoid chocolate aft

In [11]:
entry = get_sleep_diary_entry_from_memory(x.ro_memory); entry

SleepDiaryEntry(date=datetime.datetime(2023, 10, 24, 16, 43, 19, 580966), in_bed=datetime.datetime(2023, 10, 10, 0, 0), tried_to_fall_asleep=datetime.datetime(2023, 10, 30, 0, 0), time_to_fall_asleep=39, times_awake=14, time_awake=12, final_wake_up=datetime.datetime(2023, 10, 4, 0, 0), out_of_bed=datetime.datetime(2023, 10, 12, 0, 0), sleep_quality='Okay', medications='Benadryl', notes='No')

In [13]:
save_sleep_diary_entry_to_db(db_user.id, entry)

<SleepDiaryEntry: SleepDiaryEntry object>

In [10]:
list(reversed(list(enumerate(x.memory.chat_memory.messages))))

[(101,
  AIMessage(content="I'm sorry, Ivy0484, but it seems there was a hiccup while saving your sleep diary entry. Let's try that again, shall we? \n\nCould you please tell me the date of the entry? The default is yesterday's date.")),
 (100, HumanMessage(content='Summarize my sleep diary for last night')),
 (99,
  AIMessage(content="I'm sorry, Ivy0484, but it seems there was a hiccup while saving your sleep diary entry. Let's try that again, shall we? \n\nCould you please tell me the date of the entry? The default is yesterday's date.")),
 (98, HumanMessage(content='No')),
 (97,
  AIMessage(content="Thank you, Ivy0484. \n\nFinally, are there any other notes you'd like to add about your sleep last night?")),
 (96, HumanMessage(content='Benadryl')),
 (95,
  AIMessage(content='Could you please specify what medications or aids you used to help you sleep?')),
 (94, HumanMessage(content='Yes')),
 (93,
  AIMessage(content='Thank you, Ivy0484. \n\nDid you use any medications or aids to help

In [3]:
get_health_history_from_memory(x.ro_memory)

HealthHistory(date=datetime.datetime(2023, 10, 25, 0, 0), sex='Male', date_of_birth=datetime.datetime(1976, 2, 13, 0, 0), occupation='Coder', work_hours='7am - 6pm with breaks', smoking='No', alcohol='Some, mostly at weekends', physical_activity='Cyclist, surfer, weightlifter', diet='Three eggs and sausages for breakfast, salad and protein for lunch, meat and vegetables for dinner. 1g protein per lb of bodyweight.', medical_conditions='None', medications='None', family_history='None', mental_health='None', psychological_treatment='None', living_with='With family, three kids, in a house but sleep in a VW Eurovan', residence_type='House', noise_level='Very rural and quiet', goal='Sleep through the night, wake up feeling refreshed', helpful='Getting enough daylight', unhelpful='Apigenin', notes='')

In [4]:
ivy_id = "653984fa8ce7346842a9f32e"
db_user = get_user_by_id(ivy_id)

In [14]:
db_user = get_user_from_username("test")

In [15]:
db_user.to_mongo()

SON([('_id', ObjectId('653838c321f81e21a72df700')), ('name', 'Sleepy'), ('email', 'cck197@cck197.net'), ('username', 'test')])

In [18]:
DBGoalRefusal.objects(user=db_user)

[]

In [19]:
diary_entry(db_user.id)

True

In [3]:
get_collection_names()

['stress_audit',
 'goal_refusal',
 'stimulus_control_seen',
 'v_l_q_entry',
 'exercise_entry',
 'seed_pod',
 'message_store',
 'user',
 'daily_routine_seen',
 'sleep_diary_entry',
 'i_s_i_entry',
 'health_history']

In [4]:
clear_db_for_user(get_user_from_username("cck197").id)

In [4]:
db_user = get_user_from_username("test")

In [5]:
db_user.to_mongo()

SON([('_id', ObjectId('653838c321f81e21a72df700')), ('name', 'Sleepy'), ('email', 'cck197@cck197.net'), ('username', 'test')])

In [6]:
clear_db_for_user(db_user.id)

In [16]:
x.delete_chat_history(1)

In [8]:
list(reversed(list(enumerate(x.memory.chat_memory.messages))))

[]

In [6]:
x.delete_chat_history(N=4)

In [5]:
memory = x.ro_memory

In [ ]:
from 

In [8]:
entry = get_user_from_memory(memory)

In [9]:
entry

User(name='Sleepy', email='cck197@cck197.net', username='Sleepy')

In [13]:
db_entry = DBGoalRefusal(goal="xx", date=datetime.now(), user=str(db_user.id))

In [14]:
db_entry.save()

<GoalRefusal: GoalRefusal object>

In [15]:
db_entry.to_mongo()

SON([('_id', ObjectId('6538086a0cae42947a90e440')), ('user', ObjectId('653808210cae42947a90e43e')), ('date', datetime.datetime(2023, 10, 24, 11, 9, 45, 982010)), ('goal', 'xx')])

In [2]:
db_user = get_user_from_email("chris@nourishbalancethrive.com")

In [4]:
db_user.username = "cck197"

In [5]:
db_user.save()

<User: User object>

In [7]:
db_user = get_user_from_username("cck197")

get_user_from_username creating username='cck197'


In [9]:
type(db_user.id)

bson.objectid.ObjectId

In [8]:
x.db_user.delete()

In [3]:
db_user.to_mongo()

SON([('_id', ObjectId('6526d86e6015be5b3434ac13')), ('name', 'Chris'), ('email', 'chris@nourishbalancethrive.com')])

In [9]:
get_current_user().to_mongo()

SON([('_id', ObjectId('6526d86e6015be5b3434ac13')), ('name', 'Chris'), ('email', 'chris@nourishbalancethrive.com')])

In [11]:
DBUser.objects.order_by("-id").first().to_mongo()

SON([('_id', ObjectId('6537c5f8b0a6bcb7637d7e28')), ('name', 'X')])

In [2]:
x = X(hello="hello")

X() len(self.tools)=34
X.say None -> health_history
_to_args_and_kwargs self.name='get_greeting' all_args=['hello']


Hello there! I'm afraid I didn't catch your name. Could you please share it with me?

In [5]:
x("X");

Ah, splendid! Nice to meet you, X. How are you feeling today?

In [6]:
x("not bad");

_to_args_and_kwargs self.name='get_open_question' all_args=['not bad']


I'm glad to hear you're not feeling too shabby, X. It's important to keep our spirits up, isn't it? Now, would this be a good time to go through a health history questionnaire? It's a bit like a crossword puzzle, but with less confusion and more facts about you.

In [4]:
x.db_user.to_mongo()

SON([('_id', ObjectId('6537c5f8b0a6bcb7637d7e28')), ('name', 'X')])

In [6]:
x("hey");

X.say all_for_now -> diary_entry
_to_args_and_kwargs self.name='get_greeting' all_args=['hey']


Hello again, Chris! How are you feeling today? And, by the way, could I get your email address for our records? It will help us to connect offline if needed.

In [3]:
x.delete_chat_history(N=2)

In [3]:
x = X(hello=None)#, fixed_goal=True)

X() len(self.tools)=33


In [2]:
x = X();

X() len(self.tools)=33
X.say None -> seeds_entry


Splendid! Is now a good time to record a SEEDS diary entry?

In [3]:
x("not now. how does alchohol affect sleep?");

_to_args_and_kwargs self.name='get_knowledge_answer' all_args=['how does alcohol affect sleep?']


Alcohol can significantly disrupt sleep and the body's biological clock. It can affect various levels of the body's system, including the central clock and the muscle clock. The timing and dosage of alcohol consumption are crucial factors in its impact on sleep. Generally, consuming alcohol earlier in the day is better for sleep quality. 

However, the quantity of alcohol consumed also plays a significant role. Consuming less alcohol is better for sleep. For instance, it's recommended to consume no more than four units of alcohol in a day. A unit of alcohol is equivalent to the amount of alcohol in ten milliliters or eight grams of pure ethanol. 

To minimize the impact of alcohol on sleep, it's recommended to allow at least one hour before bed per unit consumed plus one additional hour. For example, if you consume two units of alcohol (equivalent to a pint of beer), you should finish drinking at least three hours before bed. This recommendation is based on the rate at which your body can detoxify alcohol. 

It's also important to note that alcohol can disrupt the biological clock at many different levels, which can lead to sleep disruptions and other health issues. Therefore, it's crucial to moderate alcohol consumption to maintain good sleep health.

In [4]:
x("ok great");

I'm glad you found the information helpful, Chris! Remember, moderation is key when it comes to alcohol consumption. If you have any more questions or if there's anything else you'd like to discuss, feel free to let me know. When you're ready to continue on your health journey, just give me a shout.

In [5]:
x("ok, ready");

Splendid! Is now a good time to record a SEEDS diary entry?

In [6]:
x("actually no. what about cold water exposure to feel more alert in the am");

_to_args_and_kwargs self.name='get_knowledge_answer' all_args=['cold water exposure to feel more alert in the morning']


Exposure to cold water in the morning can indeed make you feel more alert. This is due to the body's physiological response to sudden cold exposure, which includes a surge in heart rate, increased blood circulation, and a rush of adrenaline, all of which can result in a state of heightened alertness and energy. 

However, it's important to note that this practice may not be suitable for everyone. For individuals with certain health conditions, such as cardiovascular issues, the sudden shock of cold water could potentially pose risks. 

Here are some ways to safely incorporate cold water exposure into your morning routine:

1. Cold Showers: Start with a warm shower and gradually decrease the temperature. Over time, you can start with cooler water as your tolerance builds.

2. Face Dousing: If a full-body cold shower seems too daunting, you can start by splashing your face with cold water. This can still provide a refreshing jolt to wake you up.

3. Cold Water Immersion: For a more intense experience, consider a cold bath or even a dip in an outdoor body of water. This should be done with caution, and it's important to listen to your body's signals.

Remember, the goal is to feel invigorated and alert, not to induce unnecessary stress or discomfort. It's always important to listen to your body and adjust your routine accordingly.

In [7]:
x("ok, great");

I'm glad you found the information helpful, Chris! Remember, moderation is key when it comes to alcohol consumption. If you have any more questions or if there's anything else you'd like to discuss, feel free to let me know. When you're ready to continue on your health journey, just give me a shout.

In [8]:
x("sure, let's go");

Great! Let's start with the date of the entry. By default, we'll use today's date, which is the 23rd of October, 2023. Is that correct?

In [9]:
x("sorry, what's this?");

We're about to record a SEEDS diary entry. SEEDS stands for Sleep, Exercise, Eat, Drink, and Smile. These are the five pillars of health that we focus on. For each pillar, there are three tasks that you can do each day. We're going to go through each task and see which ones you managed to do today. Does that sound good?

In [10]:
x("Smile? really? are you sure");

Absolutely! The "Smile" pillar is all about doing things that bring you joy and happiness. It's a reminder to take time each day to do something you love, whether that's reading a book, spending time with loved ones, or even just taking a moment to appreciate a beautiful sunset. After all, mental well-being is just as important as physical health. So, did you manage to do something today that made you smile?

In [11]:
x("pretty sure SEEDS is sleep, eating, exercise, drink and STRESS");

Ah, my apologies for the confusion. You're absolutely right. The "S" in SEEDS does indeed stand for Stress, not Smile. It's about managing and reducing stress in your life. This can be achieved through various means such as meditation, deep breathing exercises, yoga, or any other activity that helps you relax and unwind. 

So, let's correct that and proceed. Did you manage to do something today to manage your stress levels?

In [12]:
x("remind me, what were my SEEDS?");

_to_args_and_kwargs self.name='get_seed_pod' all_args=['string']
get_seed_pod entry=<SeedPod: SeedPod object>


Here are the SEEDS tasks you've set:

**Sleep**:
1. Set a regular bedtime and wake-up time.
2. Create a relaxing bedtime routine, such as reading a book or taking a warm bath.
3. Make your sleep environment as comfortable as possible, e.g., use blackout curtains, earplugs, or a white noise machine.

**Exercise**:
1. Take a short walk during your lunch break.
2. Do a few stretches when you wake up in the morning.
3. Park further away from the entrance when going to the store to get a few extra steps in.

**Eating**:
1. Add an extra serving of vegetables to your dinner.
2. Swap a processed snack for a piece of fruit.
3. Drink a glass of water before each meal.

**Drinking**:
1. Replace one sugary drink per day with a glass of water.
2. Try herbal tea instead of coffee in the afternoon.
3. Drink a glass of water first thing in the morning.

**Stress**:
1. Spend 5 minutes each day practicing deep breathing.
2. Write in a gratitude journal each night before bed.
3. Take a 5-minute break every hour during work to stretch or walk around.

Now, let's go through each task and see which ones you managed to do today.

In [19]:
x.memory.clear()

In [20]:
clear_db()

In [15]:
from sleepmate.config import *

In [ ]:
SLEEPM

In [42]:
list(enumerate(self.messages))

[(0, HumanMessage(content='what should I eat')),
 (1,
  AIMessage(content='Based on the information provided, your diet should primarily consist of high-quality proteins, fiber, and minerals, while minimizing refined and concentrated carbs and fats. Here are some guidelines for what you should eat:\n\n1. **Protein Sources**: Your diet should include a few staple protein sources that you are familiar with. These could be pastured eggs, wild-caught salmon, and grass-fed beef. Other good options include ruminants (beef, lamb, bison), poultry, fish, seafood, eggs, wild game, pork, legumes, and fermented dairy (plain Greek yogurt and cottage cheese). Always opt for animals that are eating what they are supposed to be eating. For example, choose grass-fed meats, free-range organic chicken, and pastured pork when possible. \n\n2. **Carbohydrates**: If you are eating carbohydrates, make sure you have some low energy density high satiety choices available, like apples, carrots, berries, and pot

In [10]:
DBSeedsDiaryEntry.objects().order_by('-id').first().delete()

In [11]:
x.memory.clear()

In [12]:
x("what should I eat");

X.say seeds_entry -> all_for_now
_to_args_and_kwargs self.name='get_knowledge_answer' all_args=['what should I eat']


Based on the information provided, your diet should primarily consist of high-quality proteins, fiber, and minerals, while minimizing refined and concentrated carbs and fats. Here are some guidelines for what you should eat:

1. **Protein Sources**: Your diet should include a few staple protein sources that you are familiar with. These could be pastured eggs, wild-caught salmon, and grass-fed beef. Other good options include ruminants (beef, lamb, bison), poultry, fish, seafood, eggs, wild game, pork, legumes, and fermented dairy (plain Greek yogurt and cottage cheese). Always opt for animals that are eating what they are supposed to be eating. For example, choose grass-fed meats, free-range organic chicken, and pastured pork when possible. 

2. **Carbohydrates**: If you are eating carbohydrates, make sure you have some low energy density high satiety choices available, like apples, carrots, berries, and potatoes. 

3. **Emergency Protein Sources**: Keep emergency protein sources nearby. Hard-boiled eggs and individually packaged grass-fed beef sticks (without sugar) are good options. 

4. **Convenient Options**: For times when you are starving but have no time at all, consider some whey powder and a shaker bottle. 

5. **Species-Appropriate Diet**: Humans are hunter-gatherers, and we do have a species-appropriate diet. Start with a basic Paleo-ish template as a starting point. 

6. **P:E Ratio**: The P:E Diet is all about food choice. You are specifically choosing one food, over other possible food options, because it is going to provide you with a higher level of nutrient satiety—allowing you to eat less energy overall. 

7. **Seafood**: Shrimp is another delicious seafood with an amazing protein to energy ratio and plenty of nutrients, including iodine and omega-three fats. 

Remember, the key is to set yourself up for success by having these foods available as often as possible. This will make you far less likely to fall off the wagon and end up in the drive-through of your favorite fast food place.

In [3]:
clear_db(["daily_routine_seen"])

In [8]:
clear_db()

In [3]:
x = X(hello=None)

X() len(self.tools)=32


In [5]:
x.memory.save_context??

Signature: x.memory.save_context(inputs: Dict[str, Any], outputs: Dict[str, str]) -> None
Source:   
    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
        """Save context from this conversation to buffer."""
        input_str, output_str = self._get_input_output(inputs, outputs)
        self.chat_memory.add_user_message(input_str)
        self.chat_memory.add_ai_message(output_str)
File:      ~/Library/Caches/pypoetry/virtualenvs/sleepmate-kok1GflL-py3.11/lib/python3.11/site-packages/langchain/memory/chat_memory.py
Type:      method

In [3]:
diary_entry()

False

In [3]:
from datetime import datetime, time, date
midnight = datetime.combine(date.today(), time())

In [4]:
midnight

datetime.datetime(2023, 10, 22, 0, 0)

In [5]:
get_health_history_from_memory(x.ro_memory)

HealthHistory(date=datetime.datetime(2023, 10, 22, 7, 33, 37, 359605), sex='man', date_of_birth=datetime.datetime(1976, 2, 13, 0, 0), occupation='coder', work_hours='7 - 6 with breaks', smoking='nope', alcohol='some alchohol, mostly at the weekends', physical_activity="I'm a cyclist, surfer, lift weights", diet='three eggs and sausages for breakfast, salad and protein for lunch, meat and veg for dinner. 1g protein per lb of bodyweight', medical_conditions='none', medications='no', family_history='no', mental_health='no', psychological_treatment='no', living_with='live with family, three kids, in a house but sleep in a VW Eurovan', residence_type='house', noise_level="it's very rural and quiet", goal='sleep through the night, wake up feeling refreshed', helpful='getting enough daylight', unhelpful='apigenin', notes='N/A')

In [6]:
x.ro_memory.memory.return_messages

True

In [7]:
x.memory.return_messages = False

In [8]:
x.memory.buffer

"AI: what's your name?\nHuman: Chris\nAI: what's your email?\nHuman: chris@nourishbalancethrive.com\nHuman: hey\nAI: Hello Chris, it's a pleasure to meet you. I hope you're doing well. Would now be a good time to go through a health history questionnaire? It's a bit like a check-up, but without the cold stethoscope.\nHuman: not bad, a bit tired\nAI: It sounds like you're feeling a bit worn out, which is completely understandable in our fast-paced world. Let's proceed with the health history questionnaire, shall we? It might even be a nice distraction. Let's start with an easy one: what is your sex?\nHuman: man\nAI: Thank you, Chris. Now, could you please share your date of birth? Don't worry, I won't be sending any surprise birthday parties your way.\nHuman: Feb 13 1976\nAI: Thank you, Chris. You're still a spring chicken! Now, could you please tell me about your current occupation?\nHuman: coder\nAI: Ah, a coder! That's a job that requires a lot of mental gymnastics. Now, could you pl

In [4]:
x("sure");

Fantastic! Let's dive in. Remember, this is a safe space. Your answers are for your eyes only, so feel free to be as honest as possible. No need to worry about social pressures or the wagging mental fingers of should and have to. This is between you and you.

Let's start with the first domain: Family relations (other than marriage or parent). 

a) On a scale of 1-10, how important is this domain to you?
b) Again, on a scale of 1-10, how consistently have you lived in accordance with your values in this domain over the past week?

In [6]:
x.memory.clear()

In [41]:
# Provide the connection string to connect to the MongoDB database
connection_string = "mongodb://localhost:27017/"
chat_memory = MongoDBChatMessageHistory(connection_string, "test-session", database_name="sleepmate")

In [42]:
memory = ConversationBufferWindowMemory(k=1, chat_memory=chat_memory)

In [43]:
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

In [40]:
memory.chat_memory

In [16]:
memory.load_memory_variables({})

{'history': 'Human: not much you\nAI: not much'}

In [17]:
memory.messages

AttributeError: 'MyCustomMemory' object has no attribute 'collection'

> /Users/cck197/Library/Caches/pypoetry/virtualenvs/sleepmate-kok1GflL-py3.11/lib/python3.11/site-packages/langchain/memory/chat_message_histories/mongodb.py(56)messages()
     54 
     55         try:
---> 56             cursor = self.collection.find({"SessionId": self.session_id})
     57         except errors.OperationFailure as error:
     58             logger.error(error)



ipdb>  exit


In [7]:
memory = X.load_memory()

X.load_memory filename='/Users/cck197/Downloads/virtual_greg/memory.pkl'


In [7]:
memory.return_messages

True

In [5]:
get_health_history_from_memory(memory)

HealthHistory(date=datetime.datetime(2023, 10, 21, 11, 12, 24, 643083), sex='man', date_of_birth=datetime.datetime(1976, 2, 13, 0, 0), occupation='coder', work_hours='7 - 6 with breaks', smoking='no', alcohol='some alcohol, mostly at the weekends', physical_activity='cyclist, surfer, lift weights', diet='three eggs and sausages for breakfast, salad and protein for lunch, meat and veg for dinner. 1g protein per lb of bodyweight', medical_conditions='none', medications='no', family_history='no', mental_health='no', psychological_treatment='no', living_with='live with family, three kids, in a house but sleep in a VW Eurovan', residence_type='house', noise_level='very rural and quiet', goal='sleep through the night, wake up feeling refreshed', helpful='getting enough daylight', unhelpful='apigenin', notes='N/A')

In [8]:
DBGoalRefusal.objects.delete()

2

In [13]:
[o.goal for o in DBGoalRefusal.objects()]

['diary_probe', 'daily_routine']

In [ ]:
DBSleepDiaryEntry.objects(user=get_current_user()).count()

In [11]:
from sleepmate.goal import DBGoalRefusal

In [3]:
imported = import_attrs(["GOALS", "GOAL_HANDLERS"])

In [3]:
imported = import_attrs(["TOOLS"])

In [8]:
list(enumerate(memory.chat_memory.messages))

[(0, AIMessage(content="what's your name?")),
 (1, HumanMessage(content='Chris')),
 (2, AIMessage(content="what's your email?")),
 (3, HumanMessage(content='chris@nourishbalancethrive.com')),
 (4, HumanMessage(content='hey')),
 (5,
  AIMessage(content="Hello Chris! It's a pleasure to meet you. I hope you're doing well. Would now be a good time to go through a health history questionnaire? It's a bit like a check-up, but without the cold stethoscope.")),
 (6, HumanMessage(content='not bad, a bit tired')),
 (7,
  AIMessage(content="It sounds like you're feeling a bit worn out, which is completely understandable in our fast-paced world. Let's proceed with the health history questionnaire, shall we? It might even be a nice distraction. Let's start with an easy one: What is your sex?")),
 (8, HumanMessage(content='sure')),
 (9, AIMessage(content='Great! So, what is your sex?')),
 (10, HumanMessage(content='man')),
 (11,
  AIMessage(content="Thank you, Chris. Now, could you please share your

In [7]:
memory = X.load_memory()

X.load_memory filename='/Users/cck197/Downloads/virtual_greg/memory.pkl'


In [5]:
memory.chat_memory.add_user_message("yes, that's right. I got all 15 tasks done");

In [20]:
memory.chat_memory.messages = memory.chat_memory.messages[:-4]

In [21]:
X.save_memory(memory)

In [7]:
save_seeds_diary_entry(

<function sleepmate.seeds.save_seeds_diary_entry(memory: langchain.memory.readonly.ReadOnlySharedMemory, goal: str, text: str)>